In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import date_format
from pyspark.sql.types import *
import pandas as pd
from pyspark.sql.functions import unix_timestamp, from_unixtime
#import plotly.graph_objs as go
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import shutil
#init_notebook_mode(connected=True)
spark = SparkSession.builder.appName("Python Spark SQL basic example").getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
df = sqlContext.read.load('/test_dev/mba-code/dataset.csv', format='csv', header='true', inferSchema='true')
df.select("InvoiceNo", "StockCode", "Description", "Quantity", "InvoiceDate","InvoiceDateWS" ,date_format(from_unixtime(unix_timestamp('InvoiceDateWS', 'mm/dd/yyy')), 'EEEE').alias('weekday'),"CustomerID","Country").write.save("Invoices.parquet", format="parquet")
parquetFile = spark.read.parquet("Invoices.parquet")
# Parquet files can also be used to create a temporary view and then used in SQL statements.
parquetFile.createOrReplaceTempView("parquetFile")
from operator import add
from apyori import apriori, load_transactions
import pyspark.sql.functions as f
from mlxtend.preprocessing import TransactionEncoder
from pyspark.sql.functions import explode
DescriptionGrp = spark.sql("SELECT distinct InvoiceNo,Description FROM parquetFile group by InvoiceNo,Description")
print(DescriptionGrp.rdd.take(2))
minSupport = 0.05 * DescriptionGrp.rdd.count()
apr_tem=DescriptionGrp.rdd.map(lambda x: (x[0], [x[1]]).reduceByKey(lambda x,y: x + y))
transactions_mba=apr_tem.map(lambda x: (x[1]))
transactions_schema = StructType([StructField('transactions', StringType())])
transactions_df = spark.createDataFrame(transactions_mba, transactions_schema)
transactions_load = load_transactions(transactions_df)
association_rules_mba=apriori(transactions_load,min_support=0.5,min_confidence=0.7,min_lift=1.2,min_length=2)
association_rules_mba=list(association_rules_mba)

[Row(InvoiceNo=u'549044', Description=u'LUNCH BAG  BLACK SKULL.'), Row(InvoiceNo=u'549121', Description=u'GLASS CHALICE GREEN  LARGE ')]
----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 56620)
----------------------------------------


Traceback (most recent call last):
  File "/usr/lib64/python2.7/SocketServer.py", line 295, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python2.7/SocketServer.py", line 321, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python2.7/SocketServer.py", line 334, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib64/python2.7/SocketServer.py", line 649, in __init__
    self.handle()
  File "/usr/lib/python2.7/site-packages/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/usr/lib/python2.7/site-packages/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/lib/python2.7/site-packages/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/lib/python2.7/site-packages/pyspark/serializers.py", line 717, in read_int
    raise EOFError
EOFError
ERROR:root:Exception whi

Py4JError: An error occurred while calling o11.setCallSite